In [214]:
from typing import List
from functools import partial, wraps

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

## load&split

In [215]:
data = pd.read_csv('retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [216]:
users, items, interactions, max_week = data.user_id.nunique(), data.item_id.nunique(), data.shape[0], data['week_no'].nunique()

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)
print('# max week: ', max_week)

# users:  2499
# items:  89051
# interactions:  2396804
# max week:  95


In [217]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [218]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [219]:
# train/test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

## prepare result frame

In [220]:
# prepare result DataFrame
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [221]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### prepare most popular top-K and pivot matrix

In [222]:
# Топ популярных товаров
def k_popular(source, k=5000):
    """ Топ K наиболее продаваемых товаров """
    popular = source.groupby('item_id')['quantity'].sum().reset_index()
    return popular.sort_values('quantity', ascending=False).head(k).item_id.tolist()

In [223]:
top_5000 = k_popular(data_train, k=5000)

In [224]:
# флаг, что товар не из топа
top_train = data_train.copy()
top_train.loc[~top_train['item_id'].isin(top_5000), 'item_id'] = -1

top_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [225]:
user_item_matrix = pd.pivot_table(top_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix[-1] = 0        # эти товары не в топе, поэтому зануляем
user_item_matrix[user_item_matrix > 0] = 1
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [226]:
user_item_matrix.head()

item_id,-1,202291,397896,420647,480014,545926,707683,731106,818980,819063,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
# remap dictionaries
id_to_itemid = dict(enumerate(user_item_matrix.columns.values))
id_to_userid = dict(enumerate(user_item_matrix.index.values))

itemid_to_id = {v: k for k, v in id_to_itemid.items()}
userid_to_id = {v: k for k, v in id_to_userid.items()}

## 1. Baselines

### 1.1 Random recommendation

In [228]:
def random_recommendation(_items, n=5):
    """Случайные рекоммендации"""
    _items = np.array(_items)
    _recs = np.random.default_rng().choice(items, size=n, replace=False)
    return _recs.tolist()

In [229]:
%%time
items = data_train.item_id.unique()
result['random'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(3)

CPU times: user 226 ms, sys: 20.1 ms, total: 246 ms
Wall time: 246 ms


,user_id,actual,random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]"


### 1.2 Popularity-based recommendation

In [230]:
def popularity_recommendation(_data, n=5):
    """Топ-n популярных товаров"""
    popular = _data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    _recs = popular.head(n).item_id
    return _recs.tolist()

In [231]:
%%time
# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)
result['most_popular'] = result['user_id'].apply(lambda x: popular_recs)
result.head(3)

CPU times: user 105 ms, sys: 4.04 ms, total: 110 ms
Wall time: 109 ms


,user_id,actual,random,most_popular
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

In [232]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекомендации

    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    _items = items_weights['item_id'].astype(int)
    _weights = items_weights['weight'].astype(float)
    _recs = np.random.default_rng().choice(_items, size=n, replace=False, p=_weights)
    return _recs.tolist()

In [233]:
# Товары можно ранжировать не только по суммарной стоимости, но еще и по количеству проданных единиц
# т.о. в топ не попадут очень дорогие редко продаваемые товары
# с другой стороны, попадут дешевые товары, которые покупаются в больших количествах
# чтобы более-менее уравновесить эти параметры, их можно масштабировать

def calc_weights(source: pd.DataFrame, vec_fields: List[str], beta: List[float]):
    """ Расчет весов
    :param source - исходные данные
    :param vec_fields - признаки, используемые в измерении веса
    :param beta - множитель для изменения значимости признака в оценке веса
    """
    grouped_goods = source.groupby('item_id')[vec_fields].sum()
    beta = np.array(beta)

    # mean scale
    scaled_grouped_goods = ((grouped_goods - grouped_goods.mean()) / grouped_goods.std()) * beta
    # min-max scale
    # scaled_grouped_goods = (grouped_goods - grouped_goods.min()) / (grouped_goods.max() - grouped_goods.min())
    # sum scale
    # scaled_grouped_goods = grouped_goods / grouped_goods.sum()

    # в качестве веса товара берем нормализованную норму вектора товара
    vec_norma = np.linalg.norm(scaled_grouped_goods, ord=2, axis=1)
    grouped_goods['weight'] = vec_norma / vec_norma.sum()

    return grouped_goods.sort_values('weight', ascending=False).reset_index()

In [234]:
%%time
weighted_goods = calc_weights(data_train, vec_fields = ['quantity', 'sales_value'], beta=[1., 1.])
result['weighted_random'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weighted_goods, n=5))
result.head(3)

CPU times: user 2.79 s, sys: 19.8 ms, total: 2.81 s
Wall time: 2.8 s


,user_id,actual,random,most_popular,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]"


### 1.4 Random from top-K

In [235]:
%%time
result['random_from_top'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))
result.head(3)

CPU times: user 617 ms, sys: 0 ns, total: 617 ms
Wall time: 613 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]"


### 1.5 Weighted random from top-K

In [236]:
%%time
top_weighted_goods = calc_weights(data_train[data_train['item_id'].isin(top_5000)], vec_fields = ['quantity', 'sales_value'], beta=[1., 1.])
result['weighted_random_from_top'] = result['user_id'].apply(lambda x: weighted_random_recommendation(top_weighted_goods, n=5))
result.head(3)

CPU times: user 749 ms, sys: 44.1 ms, total: 793 ms
Wall time: 792 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]"


## 2. Детерминированные алгоритмы item-item

### подготовка функции-рекомендателя

In [237]:
recommender_params = {
    'user_items': sparse_user_item,
    'N': 5, # кол-во рекомендаций
    'filter_already_liked_items': False,
    'filter_items': None,
    'recalculate_user': True
}

def recommender(user_id, mdl, params):
    rec_score = mdl.recommend(userid_to_id[user_id], **params)
    return [id_to_itemid[rec[0]] for rec in rec_score]


### 2.1 Item-Item Recommender / ItemKNN

In [238]:
%%time
model = ItemItemRecommender(K=5, num_threads=0) # K - кол-во ближайших соседей
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.75 s, sys: 20.2 ms, total: 1.77 s
Wall time: 315 ms


In [239]:
%%time
result['item_knn'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 119 ms, sys: 136 µs, total: 119 ms
Wall time: 74.4 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]","[1082185, 981760, 1127831, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]","[1082185, 981760, 1098066, 826249, 995242]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]","[1082185, 981760, 1127831, 995242, 1098066]"


### 2.2 Косинусное сходство и CosineRecommender

In [240]:
%%time
model = CosineRecommender(K=5, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.79 s, sys: 15.7 ms, total: 1.8 s
Wall time: 367 ms


In [241]:
%%time
result['cosine'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 158 ms, sys: 3.87 ms, total: 162 ms
Wall time: 91.5 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]"


### 2.3 TF-IDF взвешивание и TFIDFRecommender

In [242]:
%%time
model = TFIDFRecommender(K=5, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.77 s, sys: 3.08 ms, total: 1.77 s
Wall time: 323 ms


In [243]:
%%time
result['tfidf'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 88.9 ms, sys: 14 µs, total: 88.9 ms
Wall time: 88.1 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 1098066]"


### 2.4 Трюк - по истории конкретного пользователя (K=1)

In [244]:
%%time
model = ItemItemRecommender(K=1, num_threads=0)
model.fit(sparse_user_item.T, show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.72 s, sys: 152 µs, total: 1.72 s
Wall time: 304 ms


In [245]:
%%time
result['own_purchases'] = result['user_id'].apply(partial(recommender, mdl=model, params=recommender_params))
result.head(3)

CPU times: user 61.2 ms, sys: 0 ns, total: 61.2 ms
Wall time: 60.4 ms


,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 1098066]","[1082185, 981760, 995242, 1029743, 840361]"


## 3. Измерение качества

In [246]:
result.head()

,user_id,actual,random,most_popular,weighted_random,random_from_top,weighted_random_from_top,item_knn,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[989919, 5567649, 13007818, 6704232, 7167370]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1280948, 958729, 885939, 891077, 10311479]","[6034673, 13039895, 10369057, 13007123, 6772643]","[6534178, 900842, 15778533, 1027447, 1011572]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981135, 13039154, 73681, 974561, 10355962]","[6534178, 6533889, 1029743, 6534166, 1082185]","[3025516, 6464109, 8321674, 12984700, 1114610]","[1106125, 126696, 13098159, 8358002, 10456050]","[868764, 872137, 1065122, 825994, 819227]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 1098066, 6534178, 826249, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[2407467, 13510775, 1134901, 6391210, 45372]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1060386, 13038750, 892573, 1856989]","[902806, 852514, 2845047, 15686490, 1037541]","[1108094, 975155, 6533889, 1129321, 6534178]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 878996, 1098066]","[1082185, 981760, 995242, 1029743, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[15595866, 16058630, 12384469, 1406385, 964135]","[6534178, 6533889, 1029743, 6534166, 1082185]","[992529, 13876586, 936924, 962864, 8019525]","[1058161, 1108357, 1743698, 85636, 13190266]","[839753, 1116741, 1105381, 1066289, 960142]","[1082185, 981760, 1127831, 1098066, 995242]","[1082185, 981760, 1127831, 961554, 1098066]","[1082185, 981760, 1127831, 961554, 826249]","[1082185, 995242, 1029743, 826249, 1127831]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1094619, 13008083, 10356645, 13095292, 9553080]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9245461, 1118012, 5592193, 921744, 6534178]","[7443138, 5569209, 1002691, 1113438, 827706]","[964813, 5568661, 1126420, 824005, 1051810]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 995242, 1029743, 840361]"


In [248]:
result.to_csv('predictions.csv', index=False)

Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах?

In [94]:
# your_code

In [67]:
# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k

In [68]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [69]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.0004897159647404506

In [70]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.15523996082272282

In [71]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.15122428991185113

In [72]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

0.1551420176297747

In [73]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.16092066601371205

In [74]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

/var/folders/qs/6xk5mdt9617cpq2wf0p2xkyc0000gn/T/ipykernel_37361/1284911335.py:11: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.20191740412979353